In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from sklearn.metrics import f1_score
from torch.cuda.amp import GradScaler, autocast
import time



In [3]:
# Définir les transformations pour le dataset MNIST
transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=3),  # Convertir en 3 canaux
    transforms.Resize((128, 128)),  # Redimensionner les images à 128x128 pixels
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Charger les ensembles d'entraînement et de test
train_dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
test_dataset = datasets.MNIST(root='./data', train=False, download=True, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=8, shuffle=False)

In [4]:
class PatchEmbedding(nn.Module):
    def __init__(self, img_size=128, patch_size=32, in_chans=3, embed_dim=512):
        super(PatchEmbedding, self).__init__()
        self.img_size = img_size
        self.patch_size = patch_size
        self.in_chans = in_chans
        self.embed_dim = embed_dim

        self.num_patches = (img_size // patch_size) ** 2
        self.patch_dim = in_chans * patch_size * patch_size
        self.proj = nn.Linear(self.patch_dim, embed_dim)

    def forward(self, x):
        B, C, H, W = x.shape
        x = x.reshape(B, C, H // self.patch_size, self.patch_size, W // self.patch_size, self.patch_size)
        x = x.permute(0, 2, 4, 3, 5, 1).reshape(B, (H // self.patch_size) * (W // self.patch_size), -1)
        x = self.proj(x)
        return x

In [5]:

class ViT(nn.Module):
    def __init__(self, img_size=128, patch_size=32, in_chans=3, num_classes=10, embed_dim=512, depth=4, num_heads=4, mlp_dim=1024, dropout=0.1):
        super(ViT, self).__init__()
        self.patch_embed = PatchEmbedding(img_size, patch_size, in_chans, embed_dim)
        self.cls_token = nn.Parameter(torch.zeros(1, 1, embed_dim))
        self.pos_embed = nn.Parameter(torch.zeros(1, 1 + self.patch_embed.num_patches, embed_dim))
        self.dropout = nn.Dropout(dropout)

        encoder_layer = nn.TransformerEncoderLayer(d_model=embed_dim, nhead=num_heads, dim_feedforward=mlp_dim, dropout=dropout, activation='gelu', batch_first=True)
        self.transformer = nn.TransformerEncoder(encoder_layer, num_layers=depth)

        self.norm = nn.LayerNorm(embed_dim)
        self.head = nn.Linear(embed_dim, num_classes)

    def forward(self, x):
        B = x.shape[0]
        x = self.patch_embed(x)
        cls_tokens = self.cls_token.expand(B, -1, -1)
        x = torch.cat((cls_tokens, x), dim=1)
        x = x + self.pos_embed
        x = self.dropout(x)
        x = self.transformer(x)
        x = self.norm(x[:, 0])
        x = self.head(x)
        return x


In [6]:
# Initialiser le modèle ViT
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model_vit = ViT(img_size=128, patch_size=32, in_chans=3, num_classes=10, embed_dim=512, depth=4, num_heads=4, mlp_dim=1024, dropout=0.1)
model_vit = model_vit.to(device)

# Définir la fonction de perte et l'optimiseur avec un taux d'apprentissage réduit
criterion = nn.CrossEntropyLoss()
optimizer_vit = optim.Adam(model_vit.parameters(), lr=0.00001)

# Utiliser ReduceLROnPlateau pour ajuster le taux d'apprentissage
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer_vit, 'min', patience=2, factor=0.5)

scaler = GradScaler()

<ipython-input-6-791a035daba5>:13: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = GradScaler()


In [7]:
import time  # Importer le module time pour mesurer la durée

# Fonction d'entraînement avec Mixed Precision
def train_vit_mixed_precision(model, train_loader, optimizer, criterion, scaler, scheduler, num_epochs=10):
    model.train()
    start_time = time.time()  # Début du chronomètre
    for epoch in range(num_epochs):
        running_loss = 0.0
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            with autocast():
                outputs = model(images)
                loss = criterion(outputs, labels)
            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()
            running_loss += loss.item()
        scheduler.step(running_loss / len(train_loader))  # Mettre à jour le scheduler
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}')
    total_time = time.time() - start_time  # Calcul du temps écoulé
    print(f'Training Time: {total_time:.2f} seconds')  # Afficher le temps total

# Entraîner le modèle ViT avec Mixed Precision
train_vit_mixed_precision(model_vit, train_loader, optimizer_vit, criterion, scaler, scheduler, num_epochs=10)


<ipython-input-7-6a7792d87ebd>:12: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():


Epoch [1/10], Loss: 0.7009
Epoch [2/10], Loss: 0.3282
Epoch [3/10], Loss: 0.2419
Epoch [4/10], Loss: 0.1968
Epoch [5/10], Loss: 0.1664
Epoch [6/10], Loss: 0.1447
Epoch [7/10], Loss: 0.1259
Epoch [8/10], Loss: 0.1137
Epoch [9/10], Loss: 0.1012
Epoch [10/10], Loss: 0.0912
Training Time: 1718.21 seconds


In [8]:

# Fonction d'évaluation
def evaluate_vit(model, test_loader, criterion):
    model.eval()
    correct = 0
    total = 0
    all_labels = []
    all_preds = []
    total_loss = 0

    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            loss = criterion(outputs, labels)
            total_loss += loss.item()
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            all_labels.extend(labels.cpu().numpy())
            all_preds.extend(predicted.cpu().numpy())

    accuracy = 100 * correct / total
    f1 = f1_score(all_labels, all_preds, average='weighted')
    avg_loss = total_loss / len(test_loader)
    return accuracy, f1, avg_loss

# Évaluer le modèle ViT
vit_accuracy, vit_f1, vit_loss = evaluate_vit(model_vit, test_loader, criterion)
print(f"ViT Accuracy: {vit_accuracy:.2f}%")
print(f"ViT F1 Score: {vit_f1:.4f}")
print(f"ViT Average Loss: {vit_loss:.4f}")


ViT Accuracy: 96.84%
ViT F1 Score: 0.9683
ViT Average Loss: 0.1100
